In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import time
import tracemalloc


In [ ]:
start_time = time.time()
tracemalloc.start()
train = pd.read_feather('../input/amexfeather/train_data.ftr')
train.head(5)
print("--- %s seconds ---" % (time.time() - start_time))
print(tracemalloc.get_traced_memory())

In [ ]:
train.shape

Since there are too many variables, some of them will be deleted.

### **Removal of Columns that Contains too many N/As**

In [ ]:
# Calculate percentage of N/As in columns and Rename the result dataframe
start_time = time.time()
tracemalloc.start()

nulls = pd.DataFrame(train.isnull().sum(axis = 0)* 100 / len(train)).reset_index()
nulls = nulls.rename(columns={'index':'var',0: "percent"})

# Use the columns with <30% N/As
many_nulls = nulls[nulls['percent']>70]["var"]
train.drop(columns = many_nulls,axis=1, inplace=True)

print("--- %s seconds ---" % (time.time() - start_time))
print(tracemalloc.get_traced_memory())

### **Delete Customer ID, Date & Categorical Column**

In [ ]:
#train['year'] = pd.to_datetime(train['S_2']).dt.year
#train['month'] = pd.to_datetime(train['S_2']).dt.month
#train['day'] = pd.to_datetime(train['S_2']).dt.day
train.drop(['customer_ID','S_2'],inplace=True,axis=1)



In [ ]:
cats = train.select_dtypes(include=['category'])
cats.columns
train.drop(['D_63', 'D_64', 'D_68', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117',
       'D_120', 'D_126'],inplace=True,axis=1)

In [ ]:
train = train.replace(np.nan, 0)
#train.head(1000).to_csv('./out1.csv')

## **Machine Learning Starts**

In [ ]:
#train_sample = train.head(10000)
#del train

In [ ]:
#train.head(100).to_csv('./out.csv')
#X = train_sample.loc[:, train_sample.columns!=['customer_ID','target']]
X = train.drop(['target'],inplace=False,axis=1)
y = train['target']
del train

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)


In [ ]:
from sklearn.linear_model import LogisticRegression


logit = LogisticRegression()

logit.fit( X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print("Score the X-train with Y-train is : ", round(logit.score(X_train,y_train),2))
print("Score the X-test  with Y-test  is : ", round(logit.score(X_test,y_test),2))

y_pred=logit.predict(X_test)


cm = confusion_matrix(y_test,y_pred)
print(cm)
#accuracy_score(y_test, y_test)